# Zillow Housing SQL Analysis


## by Wolfrank Guzman
@guzmanwolfrank:GitHub
#
email: guzmanwolfrank@gmail.com 



#### Objective:  Our objective is to use SQL queries to help analyze data from Zillow's Top Tier Housing Data.  Our goals are to see what changes have occured on Top Tier Home Values in all 50 states and Washington DC.  



Zillow publishes top-tier ZHVI ($, typical value for homes within the 65th to 95th percentile range for a given region) and bottom-tier ZHVI.

A user guide for this data can be found at: [Zillow](https://www.zillow.com/research/zhvi-user-guide/). 




### Import Libraries and Load Data from CSV File 

In [12]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3
import csv 
import numpy as np 
import warnings 
from pandasql import sqldf
# Ignore all warnings 
warnings.filterwarnings('ignore')


# Load your data into a DataFrame (assuming it's in a CSV file)

csv_file = r"C:\Users\Wolfrank\Desktop\Zillow.csv"
data = pd.read_csv(csv_file)
df = data


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Wolfrank\\Desktop\\Zillow.csv'

### Cleaning the Data 


After importing our libraries, let's clean the data by removing unnecessary columns.  Then, we add 1 to the Index and Size Rank columns.  We check the number of columns and display the resulting dataframe. 

In [ ]:

# Next we clean up the data in the dataframe we just created, then we save the new file.

# Data Cleaning:  Drop Columns 
columns_to_remove = ['RegionID', 'RegionType', 'StateName',]
data.drop(columns=columns_to_remove, inplace=True)

# Rename Column from RegionName to State
df.rename(columns={'RegionName': 'State'}, inplace=True)

# Add 1 to Index 
df.index = df.index + 1

# Add 1 to SizeRank 
df['SizeRank'] = df['SizeRank'] + 1

# Show number of columns
num_columns = len(data.columns)
print("Number of columns:", num_columns)

display(df)

In [ ]:
# Get the column names and join them with a comma
column_names = ','.join(df.columns)
print(column_names)


### SQL Queries 

Load the CSV data into a SQLite Database and connect to the database.  We can then write a list of SQL Queries for analysis and to generate results. 

In [ ]:
# Load the CSV data into a SQLite database
con = sqlite3.connect('zillow_data.db')

df.to_sql('zillow_data', con, if_exists='replace', index=False)

# Define SQL queries for analysis
query1 = """
SELECT State, AVG("7/31/2019") AS AvgHomeValue
FROM zillow_data
GROUP BY State
"""

query2 =  """
SELECT State, AVG("7/31/2023") AS AvgHomeValue
FROM zillow_data
GROUP BY State
"""

query3 = """
SELECT State, AVG("7/31/2023") AS AvgHomeValue
FROM zillow_data
GROUP BY State
ORDER BY AvgHomeValue DESC
LIMIT 5
"""

query4 = """
SELECT State, ("7/31/2023" - "7/31/2019") AS ChangeInValue
FROM zillow_data
ORDER BY ChangeInValue DESC
LIMIT 5
"""

query5 =  """
SELECT AVG("7/31/2023") AS average_value
FROM zillow_data
WHERE State = 'New York';
"""
query6 = """
SELECT AVG("7/31/2023") AS average_value
FROM zillow_data
WHERE State = 'New Jersey';
"""
query7 = """ 
SELECT *
FROM zillow_data
WHERE (State = 'New York' OR State = 'New Jersey')
  AND "7/31/2018" <= "7/31/2023";
"""
# Execute queries and fetch results
query_results = []

for query in [query1, query2, query3, query4, query5, query6, query7]:
    result = pd.read_sql_query(query, con)
    query_results.append(result)


### Seaborn Visualizations 

We can use the output of the SQL Queries to construct visualizations such as histograms of the data we have cleaned and queried. 

In [ ]:

# Create Seaborn charts
sns.set(style="darkgrid")

# Chart 1: Average home values by state in September 2019
plt.figure(figsize=(12, 6))
sns.barplot(data=query_results[1], x='AvgHomeValue', y='State',palette="Greys_d", alpha=0.8)
plt.title('Average Home Values by State in July 2019')
plt.xlabel('Average Home Value')
plt.ylabel('State')


In [ ]:

# Chart 2: Average home values by state in September 2021
plt.figure(figsize=(12, 6))
sns.barplot(data=query_results[0], x='AvgHomeValue', y='State',color="blue", alpha=0.5)
plt.title('Average Home Values by State in July 2023')
plt.xlabel('Average Home Value')
plt.ylabel('State')


In [ ]:
# Chart 3:  Top 5 States with Highest Average Top Tier Home Values
df = pd.read_sql_query(query3, con)
# Create a Seaborn bar plot
plt.figure(figsize=(10, 6))
sns.set(style="whitegrid")
sns.barplot(data=df, x='AvgHomeValue', y='State', color="grey", alpha=0.7)
plt.title('Top 5 States with the Highest Average Home Values (as of 7/31/2023)')
plt.xlabel('Average Home Value')
plt.ylabel('State')
plt.show()

Here we see that the state with highest change in home value between July 2019 and July 2023 was Montana. Washington came in second whil Idaho, Florida and Utah rounded out the list. 

In [ ]:

#Chart 4: Top 5  Highest Change in Top Tier Home Value 
result4 = pd.read_sql_query(query4, con)
# Create Seaborn histogram chart
sns.set(style="whitegrid")
plt.figure(figsize=(12, 6))
sns.barplot(data=result4, x='ChangeInValue', y='State', color="blue", alpha=0.6)
plt.title('Top 5 States with Highest Change in Home Values (Jul 2019 to Jul 2023)')
plt.xlabel('Change in Home Value')
plt.ylabel('State')
# Show the Charts
plt.show()


By overlaying the data from 2019 and 2023 we can see the change in price frequency at certain price points.  

This stacked count histogram shows us that there was a higher amount of mid priced homes at 200 to 400k in 2019 and that inventory has shrunken.  It has been replaced by higher priced homes amongst a smaller number of states-- as shown by the lower frequency counts. 

In [ ]:

# Chart 5: Top Tier Home Value Frequency for 2019 and 2023
df1 = pd.read_sql_query(query1, con)
df2 = pd.read_sql_query(query2, con)
# Create stacked histograms using Seaborn
plt.figure(figsize=(10, 6))
sns.histplot(df1['AvgHomeValue'], bins=20, color='grey', alpha=0.7, label='Home Avg 2019')
sns.histplot(df2['AvgHomeValue'], bins=20, color='blue', alpha=0.5, label='Home Avg 2023')
plt.title('Frequency of Top Tier Home Values for 2019 & 2023')
plt.xlabel('Average Top Tier Home Value')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In the two line chart below, we can visualize the average values in 2019 and 2023.  These values are identified by their corresponding State in the X-axis.  We can also see the sharp discrepancy between different states in terms of average value.  

In [ ]:
# Chart 6:  Average Top Tier Home Value
df1 = pd.read_sql_query(query1, con)
df2 = pd.read_sql_query(query2, con)

# Create a two-line chart using Seaborn
plt.figure(figsize=(10, 6))
sns.lineplot(data=df1, x='State', y='AvgHomeValue', label='Home Avg 2019', color='blue')
sns.lineplot(data=df2, x='State', y='AvgHomeValue', label='Home Avg 2023', color='grey')

plt.title('Avg Home Value by State (2019 vs. 2023)')
plt.xlabel('State')
plt.ylabel('AvgHomeValue')
plt.legend()
plt.grid(True)
# Rotate the X-axis text by 90 degrees
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()


Next we can take a look at specific locations.  In this instance we will Query New York and New Jersey and note any discrepancies in value. 

In [ ]:


# Define the SQL query to calculate the average
query5 = """
SELECT AVG("7/31/2023") AS average_value
FROM zillow_data
WHERE State = 'New York';
"""

# Execute the SQL query
cursor = con.execute(query5)
result = cursor.fetchone()


# Print the average value
average_value = result[0]
print(f"The average home value for New York Top Tier Homes according to Zillow on 7/31/2023 is: {average_value*1000}")


In [ ]:


# Define the SQL query to calculate the average
query6 = """
SELECT AVG("7/31/2023") AS average_value
FROM zillow_data
WHERE State = 'New Jersey';
"""

# Execute the SQL query
cursor = con.execute(query6)
result = cursor.fetchone()

# Close the database connection
#con.close()

# Print the average value
average_value = result[0]
print(f"The average home value for New Jersey Top Tier Homes according to Zillow on 7/31/2023 is: {average_value*1000}")


Below we see a SQL Query that is filtered using Pandas that shows Zillow Top Tier Home Values for the last 4 years for New York and New Jersey. 

In [ ]:
query7 = """ 
SELECT *
FROM zillow_data
WHERE (State = 'New York' OR State = 'New Jersey')
  AND "7/31/2018" <= "7/31/2023";
"""
filtered_df = pd.read_sql_query(query7, con)

display(filtered_df)

The Pandas DataFrame shows us that home values for top tier homes in New York and New Jersey are steadily rising. 

Next, let's take a look at a boxplot that shows the standard deviation of the index.  

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


df = pd.read_sql_query(query3, con)

# Create a boxplot using Seaborn
sns.set(style="whitegrid")  # Optional: Set the style for the plot

# Replace 'column_name' with the name of the column you want to create a boxplot for
sns.boxplot(data=df, x='Average Home Value')

# Optional: Customize the plot further (e.g., add labels, title, etc.)
plt.xlabel("X-axis Label")
plt.ylabel("Y-axis Label")
plt.title("Boxplot of DataFrame Column")

# Show the plot
plt.show()


## Conclusion 

We ran 7 SQL Queries using SQLite and Python to realize that home values, in Zillow's Top Tier Home Value Data have risen dramatically.  

We also learned on Chart 5, a histogram, that the amount of mid range priced homes is dwindling.  Homes across the Top Tier are becoming more expensive and concentrating in a handful of states.  

Chart 4 also taught us some new developments, surprisingly to some, Montana has had the highest overall change in Top Tier Home Value across the United States dating back to 2019. 

Overall, using SQLite, SQL Queries and Python we were able to analyze Zillow Top Tier Home Data for emerging trends and specific locations with higher values.  